<b>
<p>
<center>
<font size="5">
Quarterback College To NFL Success Model
</font>
</center>
</p>

<p>
<center>
<font size="4">
Can a model accuratley predict if a college QuarterBack will be succesful in the NFL?
</font>
</center>
</p>
    

<p>
<center>
<font size="3">
Fardin Hafiz
</font>
</center>
</p>
</b>

# Data From

https://gameonpaper.com

https://www.sports-reference.com

https://www.pro-football-reference.com

# Notes

This model is aimed to see if there is a way to used Machine Learning to predict if college quarterbacks entering the draft will be successful in the NFL. To test this I created a dataset that tracked both regular stats and advanced stats of every quarterback drafted from 2015 to 2025.I then added a target variable of "S/N" which labelled each player as a success or not in the NFL. The metric for this was if they voted for a Pro Bowl and was atleast a 3 year starter.Based on the availability of data only quarterbacks from 2015 to 2025 were used. Each players best statistical season was used for the model. Only Division 1 players statistics were available and no FCS players data was available. The quarterbacks from the 2015 to 2023 draft classes were used as the training data and the quarterbacks from 2024 and 2025 draft classes were the test data. A success probability statistic was created for the test quarterbacks and if the value was higher than 0.50 then the QB was labelled as a potential success in the NFL.

# Notebook Configuration

## Google drive

In [ ]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = "/content/drive/My Drive/Colabs Notebook/MachineLearning/NFL/"

# Get the absolute path of the shallow utilities folder
abspath_util_shallow = "/content/drive/My Drive/Colabs Notebook/MachineLearning/Utilities/"

# Get the absolute path of the shallow models folder
abspath_model_shallow = "/content/drive/My Drive/Colabs Notebook/MachineLearning/Models/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Warning

In [ ]:
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

## Matplotlib

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Set matplotlib sizes
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=20)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('legend', fontsize=20)
plt.rc('figure', titlesize=20)

## TensorFlow

In [ ]:
# The magic below allows us to use tensorflow version 2.x
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


## Random seed

In [ ]:
# The random seed
random_seed = 42

# Set random seed in tensorflow
tf.random.set_seed(random_seed)

# Set random seed in numpy
import numpy as np
np.random.seed(random_seed)

# Data Preprocessing

In [ ]:
# Change working directory to the absolute path of the shallow utilities folder
%cd $abspath_util_shallow

# Import the shallow utitilities
%run pmlm_utilities_shallow.ipynb

/content/drive/My Drive/Colabs Notebook/MachineLearning/Utilities


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(abspath_curr + "nfldata.csv")

TARGET_COL = "S/N"
YEAR_COL   = "Draft Year "

is_train  = df[YEAR_COL].between(2015, 2023) & df[TARGET_COL].notna()
is_future = df[YEAR_COL] >= 2024

meta_drop = [TARGET_COL, YEAR_COL]
if "Year" in df.columns:
    meta_drop.append("Year")

feature_cols = [c for c in df.columns if c not in meta_drop]



In [ ]:
# Print the dimension of df_train
df_train = df.loc[is_train].copy()
pd.DataFrame([[df_train.shape[0], df_train.shape[1]]],columns=['# rows', '# columns'])

,# rows,# columns
0,47,15


In [ ]:
# Print the dimension of df_test
df_test = df.loc[is_future].copy()
pd.DataFrame([[df_test.shape[0], df_test.shape[1]]], columns=['# rows', '# columns'])

,# rows,# columns
0,10,15


In [ ]:
from sklearn.model_selection import train_test_split

# Divide the training data into training (80%) and validation (20%)
df_train, df_val = train_test_split(df_train, train_size=0.8, random_state=random_seed)

# Reset the index
df_train, df_val = df_train.reset_index(drop=True), df_val.reset_index(drop=True)

In [ ]:
df_common_var = common_var_checker(df_train, df_val, df_test, target)
df_common_var

,common var
0,AvgPass
1,Cmp%
2,Draft Year
3,Dropbacks
4,EPA
5,EPA/DB
6,Int
7,Int2TD
8,Player
9,S/N


In [ ]:
# Get the features in the training data but not in the validation or test data
uncommon_feature_train_not_val_test = np.setdiff1d(df_train.columns, df_common_var['common var'])

# Print the uncommon features
pd.DataFrame(uncommon_feature_train_not_val_test, columns=['uncommon feature'])

,uncommon feature


In [ ]:
# Get the features in the validation data but not in the training or test data
uncommon_feature_val_not_train_test = np.setdiff1d(df_val.columns, df_common_var['common var'])

# Print the uncommon features
pd.DataFrame(uncommon_feature_val_not_train_test, columns=['uncommon feature'])

,uncommon feature


In [ ]:
# Get the features in the test data but not in the training or validation data
uncommon_feature_test_not_train_val = np.setdiff1d(df_test.columns, df_common_var['common var'])

# Print the uncommon features
pd.DataFrame(uncommon_feature_test_not_train_val, columns=['uncommon feature'])

,uncommon feature


In [ ]:
# Remove the uncommon features from the validation data
df_val = df_val.drop(columns=uncommon_feature_val_not_train_test)

# Remove the uncommon features from the test data
df_test = df_test.drop(columns=uncommon_feature_test_not_train_val)

# Remove the uncommon features from the training data
df_train = df_train.drop(columns=uncommon_feature_train_not_val_test)

In [ ]:
# Combine df_train, df_val and df_test
df = pd.concat([df_train, df_val, df_test], sort=False)

# Call id_checker on df
# See the implementation in pmlm_utilities.ipynb
df_id = id_checker(df)

# Print the first 5 rows of df_id
df_id.head()

,Player,Sack-Adj Yds
0,Cody Kessler,2958
1,Sean Mannion,2771
2,Christian Hackenberg,2132
3,Malik Willis,2433
4,Justin Fields,2667


In [ ]:
import numpy as np

# Remove identifiers from df_train
df_train.drop(columns=np.intersect1d(df_id.columns, df_train.columns), inplace=True)

# Remove identifiers from df_val
df_val.drop(columns=np.intersect1d(df_id.columns, df_val.columns), inplace=True)

# Remove identifiers from df_test
df_test.drop(columns=np.intersect1d(df_id.columns, df_test.columns), inplace=True)

In [ ]:
# Get the date time variables
datetime_vars = []

# Call datetime_transformer on df_train
df_train = datetime_transformer(df_train, datetime_vars)

# Call datetime_transformer on df_val
df_val = datetime_transformer(df_val, datetime_vars)

# Call datetime_transformer on df_test
df_test = datetime_transformer(df_test, datetime_vars)

In [ ]:
# Print the unique data type of variables with NaN
pd.DataFrame(df_nan['dtype'].unique(), columns=['dtype'])

,dtype
0,float64


In [ ]:
# Get the variables with missing values, their proportion of missing values and data type
df_miss = df_nan[df_nan['dtype'] == 'float64'].reset_index(drop=True)

# Print df_miss
df_miss

,var,proportion,dtype
0,S/N,0.175439,float64


In [ ]:
# Separating the training data
df_train = df.iloc[:df_train.shape[0], :]

# Separating the validation data
df_val = df.iloc[df_train.shape[0]:df_train.shape[0] + df_val.shape[0], :]

# Separating the test data
df_test = df.iloc[df_train.shape[0] + df_val.shape[0]:, :]

In [ ]:
# Combine df_train, df_val and df_test
df = pd.concat([df_train, df_val, df_test], sort=False)

# Print the unique data type of variables in df
pd.DataFrame(df.dtypes.unique(), columns=['dtype'])

,dtype
0,object
1,float64
2,int64


In [ ]:
ID_EXCLUDE = ["Player "]

cats_to_encode = np.setdiff1d(
    np.intersect1d(df.columns, df_cat['var']),
    [target] + [c for c in ID_EXCLUDE if c in df.columns]
)

df_cat = cat_var_checker(df)

df_cat

,var,nunique
0,Player,57


In [ ]:
df.columns = df.columns.str.strip()

df_cat = cat_var_checker(df)

cats_to_encode = [c for c in df_cat['var'] if c not in [target, "Player"]]

df = pd.get_dummies(df, columns=cats_to_encode)

df.head()

,Player,AvgPass,Cmp%,TD,Dropbacks,Sack-Adj Yds,EPA,Yds/DB,EPA/DB,SR,Int,Int2TD,S/N,year,Draft Year
0,Cody Kessler,252.6,66.8,29,448,2958,54.45,6.60,0.12,45.1,7,4.14,0.0,2015,2016
1,Sean Mannion,263.7,62.3,15,481,2771,-43.25,5.76,-0.09,39.9,8,1.88,0.0,2014,2015
2,Christian Hackenberg,194.2,53.5,16,384,2132,-45.75,5.55,-0.12,34.6,6,2.67,0.0,2015,2016
3,Malik Willis,220.2,61.1,27,371,2433,-1.20,6.56,0.00,39.9,12,2.25,0.0,2021,2022
4,Justin Fields,233.8,67.2,41,334,2667,132.59,7.99,0.40,54.2,3,13.67,0.0,2019,2021


In [ ]:
id_cols = ["Player", "Draft Year", "year"]

X_train = X_train.drop(columns=id_cols, errors="ignore")
X_val   = X_val.drop(columns=id_cols, errors="ignore")
X_test  = X_test.drop(columns=id_cols, errors="ignore")


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

n_train, n_val = len(df_train), len(df_val)

df_train = df.iloc[:n_train, :].copy()
df_val   = df.iloc[n_train:n_train+n_val, :].copy()
df_test  = df.iloc[n_train+n_val:, :].copy()

df_train[target] = le.fit_transform(df_train[target].astype(str))
df_val[target]   = le.transform(df_val[target].astype(str))

df = pd.concat([df_train, df_val, df_test], sort=False)

In [ ]:
n_train, n_val = len(df_train), len(df_val)

# Re-split
df_train = df.iloc[:n_train, :].copy()
df_val   = df.iloc[n_train:n_train+n_val, :].copy()
df_test  = df.iloc[n_train+n_val:, :].copy()

In [ ]:
# Feature matrices (keep as DataFrames!)
X_train = df_train.drop(columns=[target]).copy()
X_val   = df_val.drop(columns=[target]).copy()
X_test  = df_test.drop(columns=[target]).copy()

# Targets
y_train = df_train[target].values
y_val   = df_val[target].values


In [ ]:
from sklearn.preprocessing import StandardScaler

num_cols = X_train.select_dtypes(include=["int64", "float64"]).columns

# Initialize scaler
ss = StandardScaler()

# Fit on train numeric cols
X_train[num_cols] = ss.fit_transform(X_train[num_cols])

# Transform val/test with the same scaler
X_val[num_cols]  = ss.transform(X_val[num_cols])
X_test[num_cols] = ss.transform(X_test[num_cols])


# Hyperparameter Tuning

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

models = {'lr': LogisticRegression(class_weight='balanced', random_state=random_seed),
          'mlpc': MLPClassifier(early_stopping=True, random_state=random_seed),
          'rfc': RandomForestClassifier(class_weight='balanced', random_state=random_seed),
          'hgbc': HistGradientBoostingClassifier(random_state=random_seed)}

In [ ]:
from sklearn.pipeline import Pipeline

from sklearn.pipeline import Pipeline

pipes = {}

for acronym, model in models.items():
    pipes[acronym] = Pipeline([('model', model)])

In [ ]:
X_train_val, y_train_val, ps = get_train_val_ps(X_train, y_train, X_val, y_val)

In [ ]:
param_grids = {}

param_grids['lr'] = [{
    'model__C': [0.05, 0.1, 0.5],
    'model__penalty': ['l2'],
    'model__solver': ['lbfgs'],
    'model__max_iter': [1000]
}]

param_grids['mlpc'] = [{
    'model__hidden_layer_sizes': [(16,), (32,)],
    'model__alpha': [1e-3, 1e-2],
    'model__learning_rate_init': [1e-3, 3e-4],
    'model__activation': ['relu'],
    'model__solver': ['adam'],
    'model__early_stopping': [True],
    'model__batch_size': ['auto'],
    'model__max_iter': [1000]
}]

param_grids['rfc'] = [{
    'model__n_estimators': [200],
    'model__max_depth': [3, 5],
    'model__min_samples_leaf': [5, 10],
    'model__max_features': ['sqrt'],
    'model__class_weight': ['balanced']
}]

param_grids['hgbc'] = [{
    'model__learning_rate': [0.05, 0.1],
    'model__max_depth': [3],
    'model__min_samples_leaf': [5, 10],
    'model__l2_regularization': [0.0, 1.0],
    'model__max_iter': [100],
}]


In [ ]:
target = "S/N"
id_cols = ["Player", "Draft Year", "year", "Team", "College"]

# Training
y_train = df_train[target].copy()
X_train = df_train.drop(columns=id_cols + [target], errors="ignore")

# Validation
y_val = df_val[target].copy()
X_val = df_val.drop(columns=id_cols + [target], errors="ignore")

# Future test (no labels)
X_test = df_test.drop(columns=id_cols + [target], errors="ignore")

# Combine train + val for grid search
from sklearn.model_selection import PredefinedSplit
import numpy as np

X_train_val = pd.concat([X_train, X_val])
y_train_val = np.concatenate([y_train, y_val])

# tell GridSearchCV which rows belong to val
test_fold = [-1]*len(X_train) + [0]*len(X_val)
ps = PredefinedSplit(test_fold)


In [ ]:
from sklearn.model_selection import GridSearchCV

# The list of [best_score_, best_params_, best_estimator_] obtained by GridSearchCV
best_score_params_estimator_gs = []

# For each model
for acronym in pipes.keys():
    # GridSearchCV
    gs = GridSearchCV(estimator=pipes[acronym],
                      param_grid=param_grids[acronym],
                      scoring='f1_macro',
                      n_jobs=2,
                      cv=ps,
                      return_train_score=True)

    # Fit the pipeline
    gs = gs.fit(X_train_val, y_train_val)

    # Update best_score_params_estimator_gs
    best_score_params_estimator_gs.append([gs.best_score_, gs.best_params_, gs.best_estimator_])

    # Sort cv_results in ascending order of 'rank_test_score' and 'std_test_score'
    cv_results = pd.DataFrame.from_dict(gs.cv_results_).sort_values(by=['rank_test_score', 'std_test_score'])

    # Get the important columns in cv_results
    important_columns = ['rank_test_score',
                         'mean_test_score',
                         'std_test_score',
                         'mean_train_score',
                         'std_train_score',
                         'mean_fit_time',
                         'std_fit_time',
                         'mean_score_time',
                         'std_score_time']

    # Move the important columns ahead
    cv_results = cv_results[important_columns + sorted(list(set(cv_results.columns) - set(important_columns)))]

    # Write cv_results file
    cv_results.to_csv(path_or_buf=abspath_curr + '/result/submission/' + acronym + '.csv', index=False)

# Sort best_score_params_estimator_gs in descending order of the best_score_
best_score_params_estimator_gs = sorted(best_score_params_estimator_gs, key=lambda x : x[0], reverse=True)

# Print best_score_params_estimator_gs
pd.DataFrame(best_score_params_estimator_gs, columns=['best_score', 'best_param', 'best_estimator'])

,best_score,best_param,best_estimator
0,0.696970,"{'model__C': 0.05, 'model__max_iter': 1000, 'm...","(LogisticRegression(C=0.05, class_weight='bala..."
1,0.696970,"{'model__class_weight': 'balanced', 'model__ma...","((DecisionTreeClassifier(max_depth=3, max_feat..."
2,0.696970,"{'model__l2_regularization': 1.0, 'model__lear...",(HistGradientBoostingClassifier(l2_regularizat...
3,0.333333,"{'model__activation': 'relu', 'model__alpha': ...","(MLPClassifier(alpha=0.001, early_stopping=Tru..."


In [ ]:
# Get the best_score, best_params and best_estimator obtained by GridSearchCV
best_score_gs, best_params_gs, best_estimator_gs = best_score_params_estimator_gs[0]

In [ ]:
best_model = best_score_params_estimator_gs[0][2]
future_probs = best_model.predict_proba(X_test)[:, 1]
future_pred  = (future_probs >= 0.5).astype(int)

df_future_out = df_test.copy()
df_future_out["pred_prob_success"] = future_probs
df_future_out["pred_success_yn"]   = np.where(future_pred == 1, "Yes", "No")

df_future_out = df_future_out[["Player", "pred_prob_success", "pred_success_yn"]]

print(df_future_out)



             Player  pred_prob_success pred_success_yn
47   Caleb Williams           0.379089              No
48   Jayden Daniels           0.618390             Yes
49       Drake Maye           0.799698             Yes
50    Micheal Penix           0.931434             Yes
51      JJ McCarthy           0.815275             Yes
52           Bo Nix           0.830380             Yes
53         Cam Ward           0.732776             Yes
54      Jaxson Dart           0.293995              No
55     Tyler Shough           0.328887              No
56  Dillion Gabriel           0.857082             Yes
